In [2]:
import numpy as np
import pandas as pd
from datetime import datetime 
from datetime import timedelta
import csv 
import matplotlib.pyplot as plt
from datetime import date
from datetime import time
import calendar
import locale
import seaborn as sns

In [3]:
def convertTimeStamp(string):
    """Converts a string to a datetime object.
    """
    time = datetime.strptime(string,'%m/%d/%Y %I:%M:%S %p') #Accepts time string, converts it to datetime object.
    return time

In [4]:
#convert seconds to weeks, days, hours, minutes, seconds

intervals = (
    ('weeks', 604800),  # 60 * 60 * 24 * 7
    ('days', 86400),    # 60 * 60 * 24
    ('hours', 3600),    # 60 * 60
    ('minutes', 60),
    ('seconds', 1),
    )

def display_time(seconds, granularity=2):
    result = []

    for name, count in intervals:
        value = seconds // count
        if value:
            seconds -= value * count
            if value == 1:
                name = name.rstrip('s')
            result.append("{} {}".format(value, name))
    return ', '.join(result[:granularity])

In [5]:
def elapsedInMonths(filepath):
    """councilTime accepts a csv file and returns a dataframe which displays the average elapsed time between created date 
        and update date for service requests in each distinct council district.
        Data from: https://data.lacity.org/A-Well-Run-City/MyLA311-Service-Request-Data-2018/h65r-yf5i
        """
    
    dataframe = pd.read_csv(filepath)
    
    
    #filter dateset to CreatedDate and UpdatedDate columns
    created_df = dataframe['CreatedDate'].dropna().apply(convertTimeStamp, 0) #apply convertTimeStamp to CreatedDate
    updated_df = dataframe['UpdatedDate'].dropna().apply(convertTimeStamp, 0) #apply convertTimeStamp to UpdatedDate
    
    
    #create elapsed time and elapsed seconds columns and insert into dataframe
    elapsed_df = updated_df - created_df #calculate the difference in the values in CreatedDate and UpdatedDate
    dataframe.insert(3, "ElapsedTime", elapsed_df) #insert column ElapsedTime with dataframe elapsed_df into dataframe dataframe
    elapsed_seconds = dataframe['ElapsedTime'].dt.total_seconds() #convert ElapsedTime values to seconds
    dataframe.insert(4, 'ElapsedTime in Seconds', elapsed_seconds)
    
    
    #convert CreatedDate and UpdatedDate to datetime objects
    created_df = pd.to_datetime(dataframe['CreatedDate'], format='%m/%d/%Y %I:%M:%S %p')
    updated_df = pd.to_datetime(dataframe['UpdatedDate'], format='%m/%d/%Y %I:%M:%S %p')
    
    
    df4 = dataframe[['CreatedDate', 'UpdatedDate', 'ElapsedTime', 'ElapsedTime in Seconds']] #create a dataframe df4 with dates and elapsed time columns from dataframe
    df4 = df4.sort_values(by = 'CreatedDate')
        
    
    #create a dataframe of the months of CreatedDate rows
    for row in range(0, len(created_df)-1):
        created_df[row] = created_df[row].month
        
    
    #create dataframes for each unique month with ElapsedTime column
    monthsList = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    for num in range(0,11):
        month = monthsList[num]
        print('Elapsed Times in', month)
        
        #group CreatedDate by month
        is_month =  created_df==num+1
        df4_month = df4[is_month]
        
        #print columns CreatedDate, UpdatedDate, ElapsedTime, ElapsedTime in Seconds
        month_time = df4_month[['CreatedDate', 'UpdatedDate', 'ElapsedTime']]
        print(month_time)

        #find average elapsed time for each month
        elapsed_avg = df4_month['ElapsedTime in Seconds'].mean()
        elapsed_avg = display_time(elapsed_avg, 5)

        print('\nAverage Elapsed Time in', monthsList[num],':', elapsed_avg)
        print('\n\n')
        
    

In [6]:
elapsedInMonths(r"C:\Users\hanaa\Downloads\MyLA311_Service_Request_Data_2018.csv")

C:\Users\hanaa\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Elapsed Times in January
                  CreatedDate             UpdatedDate     ElapsedTime
395    01/01/2018 01:00:00 PM  01/02/2018 03:02:00 PM 1 days 02:02:00
396    01/01/2018 01:00:00 PM  01/04/2018 10:06:00 AM 2 days 21:06:00
400    01/01/2018 01:01:00 PM  01/05/2018 01:29:00 PM 4 days 00:28:00
399    01/01/2018 01:01:00 PM  01/04/2018 08:07:00 AM 2 days 19:06:00
398    01/01/2018 01:01:00 PM  01/03/2018 12:15:00 PM 1 days 23:14:00
...                       ...                     ...             ...
99499  01/31/2018 12:59:00 PM  01/31/2018 12:59:00 PM 0 days 00:00:00
99500  01/31/2018 12:59:00 PM  01/31/2018 12:59:00 PM 0 days 00:00:00
99501  01/31/2018 12:59:00 PM  01/31/2018 12:59:00 PM 0 days 00:00:00
99502  01/31/2018 12:59:00 PM  01/31/2018 12:59:00 PM 0 days 00:00:00
99504  01/31/2018 12:59:00 PM  02/07/2018 01:49:00 PM 7 days 00:50:00

[101409 rows x 3 columns]


C:\Users\hanaa\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.



Average Elapsed Time in January : 4.0 days, 16.0 hours, 18.0 minutes, 23.0 seconds



Elapsed Times in February
                   CreatedDate             UpdatedDate     ElapsedTime
103167  02/01/2018 01:00:00 PM  02/02/2018 01:19:00 PM 1 days 00:19:00
103165  02/01/2018 01:00:00 PM  02/05/2018 12:00:00 PM 3 days 23:00:00
103166  02/01/2018 01:00:00 PM  02/02/2018 10:36:00 AM 0 days 21:36:00
103168  02/01/2018 01:01:00 PM  02/02/2018 02:12:00 PM 1 days 01:11:00
103169  02/01/2018 01:01:00 PM  02/02/2018 10:38:00 AM 0 days 21:37:00
...                        ...                     ...             ...
184927  02/28/2018 12:58:00 PM  03/02/2018 10:41:00 AM 1 days 21:43:00
184935  02/28/2018 12:59:00 PM  03/08/2018 02:48:00 PM 8 days 01:49:00
184932  02/28/2018 12:59:00 PM  03/01/2018 03:49:00 PM 1 days 02:50:00
184933  02/28/2018 12:59:00 PM  03/04/2018 01:13:00 PM 4 days 00:14:00
184934  02/28/2018 12:59:00 PM  03/07/2018 04:32:00 PM 7 days 03:33:00

[85276 rows x 3 columns]

Average 